# Real-world network

In [1]:
import networkx as nx
from networkx.algorithms.centrality import *
import numpy as np
import operator
from analysis import *

In [2]:
g = nx.read_gml("./RealNetworks/ego_facebook.gml")

In [3]:
print(nx.info(g))

Graph with 4039 nodes and 88234 edges


<ipython-input-3-99dac2172e06>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))


# Check connectedness

In [4]:
nx.is_connected(g)

True

In [5]:
nx.number_connected_components(g)

1

# Get LCC if graph is disconnected

# Infer community structure with Louvain

In [6]:
nx.__version__

'2.8.4'

In [7]:
import pandas as pd
from os.path  import exists
import infomap as im

In [8]:
im.__version__

'2.3.0'

In [9]:
G = g

In [10]:
partition   = nx.algorithms.community.louvain_communities(G, seed = 0)
partition_Q = nx.algorithms.community.modularity(G, partition)

for seed in Bar("Progress", check_tty = False).iter(range(1,5)):
    partition_   = nx.algorithms.community.louvain_communities(G, seed = seed)
    partition_Q_ = nx.algorithms.community.modularity(G, partition_)
    
    if partition_Q_ > partition_Q:
        partition   = partition_
        partition_Q = partition_Q_

Progress |################################| 4/4


In [11]:
communities = { community_ID : set(community) for (community_ID, community) in enumerate(partition, start = 1)}

In [12]:
community_assignments = dict()
for (community_ID, community) in communities.items():
    for node in community:
        community_assignments[node] = community_ID

In [13]:
results_file = "./results/ego-facebook-modularity.csv"

if not exists(results_file):
    # Map Equation Centrality
    tree_file = "./results/ego-facebook-temp.tree"

    # use Infomap to write a tree file that we change to reflect the Louvain partition
    infomap = im.Infomap(silent = True, num_trials = 1, directed = False, two_level = True, seed = 42)
    infomap.add_networkx_graph(G)
    infomap.run()
    infomap.write_tree(tree_file)
    
    infomap_tree = dict()

    # read the tree file
    with open(tree_file, "r") as fh:
        for line in fh:
            if not line.startswith("#"):
                _, flow, name, node_id = line.strip().split(" ")
                name = name[1:-1]
                infomap_tree[int(name)] = (flow, node_id)
    
    # write the Louvain partition to an Infomap tree file
    with open("./results/ego-facebook-modularity.tree", "w") as fh:
        for community_ID, community in enumerate(partition, start = 1):
            for node_pos, node in enumerate(community, start = 1):
                path = f"{community_ID}:{node_pos}"
                flow, node_id = infomap_tree[node]
                fh.write(f"{path} {flow} \"{node}\" {node_id}\n")
    
    # Load the Louvain partition with Infomap and use unrecorded link teleportation
    infomap2 = im.Infomap("--silent --no-infomap --cluster-data ./results/ego-facebook-modularity.tree")
    node_mapping = infomap2.add_networkx_graph(G)
    infomap2.run()
    
    mec_ranks = [node for node, _score in sorted( [ (node_mapping[node.node_id], node.modular_centrality) for node in infomap2.iterLeafNodes() ]
                                                 , key     = second
                                                 , reverse = True
                                                 )]

    
    index = []
    for node in G.nodes:
        index.append(node)

    results = pd.DataFrame(index=index)

    results["map equation centrality rank"]       = toRanking(mec_ranks,        results.index)
    results.to_csv(results_file)

else:
    results = pd.read_csv(results_file, index_col = 0)

    mec_ranks         = mkRanks(results["map equation centrality rank"])

In [14]:
len(mec_ranks)

4039

In [15]:
# node 107 has rank 0 (i.e. 1)
# node 1684 has rank 1 (i.e. 2)
# ...
# node 0 has rank 4 (i.e. 5)
mec_ranks

['107',
 '1684',
 '1912',
 '3437',
 '0',
 '2543',
 '2347',
 '1888',
 '1800',
 '1663',
 '1352',
 '2266',
 '483',
 '348',
 '1730',
 '1985',
 '1941',
 '2233',
 '2142',
 '1431',
 '1199',
 '1584',
 '2206',
 '1768',
 '2611',
 '2229',
 '2410',
 '1086',
 '1589',
 '2218',
 '2047',
 '2078',
 '1993',
 '2123',
 '1746',
 '2464',
 '1827',
 '2560',
 '2240',
 '2507',
 '2244',
 '2309',
 '1983',
 '1126',
 '2088',
 '2602',
 '2340',
 '2131',
 '2369',
 '2324',
 '2590',
 '2604',
 '2542',
 '1804',
 '2220',
 '2073',
 '2607',
 '2188',
 '1390',
 '2059',
 '2172',
 '1943',
 '2150',
 '1833',
 '2428',
 '1946',
 '2526',
 '1377',
 '1612',
 '1917',
 '2201',
 '2331',
 '2601',
 '1621',
 '1938',
 '2090',
 '2564',
 '2624',
 '2118',
 '2384',
 '1714',
 '2030',
 '1962',
 '2586',
 '1559',
 '2064',
 '2625',
 '2184',
 '1707',
 '2593',
 '2615',
 '2275',
 '2103',
 '2290',
 '2111',
 '1078',
 '2414',
 '2323',
 '1835',
 '1376',
 '1613',
 '2600',
 '2339',
 '2500',
 '2104',
 '2271',
 '2326',
 '1610',
 '2395',
 '2376',
 '1399',
 '1984'

In [30]:
counter = len(mec_ranks)
dict_map_eq_centrality = {}

In [31]:
for node in mec_ranks:
    dict_map_eq_centrality[int(node)] = counter
    counter = counter - 1

In [32]:
dict_map_eq_centrality

{107: 4039,
 1684: 4038,
 1912: 4037,
 3437: 4036,
 0: 4035,
 2543: 4034,
 2347: 4033,
 1888: 4032,
 1800: 4031,
 1663: 4030,
 1352: 4029,
 2266: 4028,
 483: 4027,
 348: 4026,
 1730: 4025,
 1985: 4024,
 1941: 4023,
 2233: 4022,
 2142: 4021,
 1431: 4020,
 1199: 4019,
 1584: 4018,
 2206: 4017,
 1768: 4016,
 2611: 4015,
 2229: 4014,
 2410: 4013,
 1086: 4012,
 1589: 4011,
 2218: 4010,
 2047: 4009,
 2078: 4008,
 1993: 4007,
 2123: 4006,
 1746: 4005,
 2464: 4004,
 1827: 4003,
 2560: 4002,
 2240: 4001,
 2507: 4000,
 2244: 3999,
 2309: 3998,
 1983: 3997,
 1126: 3996,
 2088: 3995,
 2602: 3994,
 2340: 3993,
 2131: 3992,
 2369: 3991,
 2324: 3990,
 2590: 3989,
 2604: 3988,
 2542: 3987,
 1804: 3986,
 2220: 3985,
 2073: 3984,
 2607: 3983,
 2188: 3982,
 1390: 3981,
 2059: 3980,
 2172: 3979,
 1943: 3978,
 2150: 3977,
 1833: 3976,
 2428: 3975,
 1946: 3974,
 2526: 3973,
 1377: 3972,
 1612: 3971,
 1917: 3970,
 2201: 3969,
 2331: 3968,
 2601: 3967,
 1621: 3966,
 1938: 3965,
 2090: 3964,
 2564: 3963,
 2624

In [33]:
# Write Centrality
with open('./MapEquationCentralityResults/OSN-EgoFacebook/dict_map_eq_centrality.txt', 'w') as f:
    for key, value in dict_map_eq_centrality.items():
        f.write('%s:%s\n' % (key, value))

In [34]:
# Read Centrality
dict_map_eq_centrality_read = dict()
with open('./MapEquationCentralityResults/OSN-EgoFacebook/dict_map_eq_centrality.txt') as raw_data:
    for item in raw_data:
        if ':' in item:
            key,value = item.split(':', 1)
            value = value.replace('\n', '')
            dict_map_eq_centrality_read[int(key)]=float(value)
        else:
            pass # deal with bad lines of text here

In [35]:
dict_map_eq_centrality_read == dict_map_eq_centrality

True